# One-Dimensional Flow and Comparison to Analytical Solution

Use DIS2D (overland flow) and DISV1D (Channel Flow) to simulate one-dimensional flow and compare the results to the analytical solution.

In [ ]:
import pathlib as pl
import matplotlib.pyplot as plt
import numpy as np
import flopy
from build_simulation import build_simulation, make_plot, print_flow_comparison

In [ ]:
ws = pl.Path('./mf6_sim')
name = 'oned'

In [ ]:
def h_analytical(x, x0, x1, h0, h1):
    rho = (x - x0) / (x1 - x0)
    h = ((1 - rho) * h0 ** (13./3.) + rho * h1 ** (13./3.)) ** (3./13.)
    return h

def calc_flow(x0, x1, h0, h1, roughness):
    q = (1. / roughness) * np.sqrt(3. / 13. * (h0 ** (13/3) - h1 ** (13/3)) / (x1 - x0))
    return q

In [ ]:
# make modflow model
exe_name = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/mf6"
sim_ws = ws

Lx = 110000
ncol = 501
dx = Lx / ncol

roughness = 1.0
h0 = 10.0
h1 = 1.0
x0 = 0.5 * dx
x1 = Lx - 0.5 * dx
x = np.linspace(x0, x1, ncol)
h_analytical_solution = h_analytical(x, x0, x1, h0, h1)

# numerical settings
dev_swr_conductance = False
central_in_space = False
save_velocity = False

if False:
    fig = plt.figure(figsize=(6, 4))
    ax = plt.subplot(1, 1, 1)
    ax.plot(x, h_analytical_solution, 'bo-')
    ax.set_xlim(0, Lx)
    ax.set_ylim(0, h0)
    ax.set_xlabel("X-distance, in meters")
    ax.set_ylabel("Stage, in meters")
    ax.set_title("Analytical Solution")

In [ ]:
# Create, write, and run simulation
sim = build_simulation(
    sim_ws,
    exe_name,
    ncol,
    dx,
    h0,
    h1,
    roughness,
    dev_swr_conductance,
    central_in_space,
    save_velocity,
)
sim.write_simulation()
sim.run_simulation()

In [ ]:
channel_symbol = {
    "marker": "o",
    "mfc": "none",
    "mec": "red",
    "ls": "none",
    "label": "Channel (DISV1D)",
}
channel_error_symbol = {
    "marker": None,
    "color": "red",
    "ls": "dashed",
    "linewidth": 1.0,
}
overland_symbol = {
    "marker": "^",
    "mec": "blue",
    "mfc": "none",
    "ls": "none",
    "label": "Overland (DIS2D)",
}
overland_error_symbol = {
    "marker": None,
    "color": "blue",
    "ls": "dotted",
    "linewidth": 1.0,
}
symbols = {
    "channel": channel_symbol, 
    "overland": overland_symbol,
    "channel_error": channel_error_symbol, 
    "overland_error": overland_error_symbol
}
extent = [0, Lx, 0, h0 + 1]
istep = 10
fig, ax = make_plot(sim, x, h_analytical_solution, extent, istep, symbols)
fig.savefig("../../doc/figures/oned.png", dpi=300)
fig

In [ ]:
q_analytical = calc_flow(x0, x1, h0, h1, roughness)
print_flow_comparison(sim, q_analytical)

# Specified Inflow Version

In [ ]:
def h_analytical_specified_inflow(Q0, rough, x, x1, h1):
    h = (13./3. * Q0 ** 2 * rough ** 2 * (x1 - x) + h1 ** (13./ 3.)) ** (3./ 13.)
    return h

In [ ]:
# Lx = 110000
# ncol = 101
# dx = Lx / ncol

# Q0 = 10.0 # m^2/s
# rough = 1.0
# h1 = 1.0
# x0 = 0.5 * dx
# x1 = Lx - 0.5 * dx
# x = np.linspace(x0, x1, ncol)
Q0 = q_analytical
h_analytical2 = h_analytical_specified_inflow(q_analytical, roughness, x, x1, h1)

if False:
    fig = plt.figure(figsize=(6, 4))
    ax = plt.subplot(1, 1, 1)
    ax.plot(x, h_analytical2, 'b-')
    ax.set_xlim(0, Lx)
    ax.set_ylim(0, h_analytical2.max())
    ax.set_xlabel("X-distance, in meters")
    ax.set_ylabel("Stage, in meters")
    ax.set_title("Analytical Solution")
    fig

In [ ]:
# Create, write, and run simulation
sim = build_simulation(
    sim_ws,
    exe_name,
    ncol,
    dx,
    h0,
    h1,
    roughness,
    dev_swr_conductance,
    central_in_space,
    save_velocity,
    Q0=Q0
)
sim.write_simulation()
sim.run_simulation()

In [ ]:
extent = [0, Lx, 0, h0 + 1]
istep = 10
fig, ax = make_plot(sim, x, h_analytical2, extent, istep, symbols)
# fig.savefig("../../doc/figures/oned.png", dpi=300)
fig

In [ ]:
print_flow_comparison(sim, q_analytical)